In [1]:
import pandas as pd
import numpy as np

In [2]:
path = r"C:\Users\Apurav\Downloads\hackTheFuture\Hack-the-Future\datasets\crypto_dataset_large2.csv"

df = pd.read_csv(path)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
# df['algorithm_label'] = label_encoder.fit_transform(df['Algorithm'])

# Train-Test Split
# X_train, X_test, y_train, y_test = train_test_split(df['Ciphertext'], df['algorithm_label'], test_size=0.2, random_state=42)


In [4]:
df["Algorithm"].value_counts()

Algorithm
DES         20000
ChaCha20    20000
ECC         20000
Blowfish    20000
RC4         20000
Name: count, dtype: int64

In [5]:
selected_categories = ["AES", "3DES", 
                       "RSA", "SHA-256"]
df_filtered = df[~df['Algorithm'].isin(selected_categories)]

In [6]:
df_filtered["Algorithm"].value_counts()

Algorithm
DES         20000
ChaCha20    20000
ECC         20000
Blowfish    20000
RC4         20000
Name: count, dtype: int64

In [7]:
df_filtered['algorithm_label'] = label_encoder.fit_transform(df_filtered['Algorithm'])

In [8]:
df_filtered["algorithm_label"].value_counts()

algorithm_label
2    20000
1    20000
3    20000
0    20000
4    20000
Name: count, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df_filtered['Ciphertext'], df_filtered['algorithm_label'], test_size=0.3, random_state=42)

# import hashlib
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Feature extraction using character frequency and hashing
# def extract_features(cipher_texts):
#     vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1))
#     return vectorizer.fit_transform(cipher_texts)

# X_train_features = extract_features(X_train)
# X_test_features = extract_features(X_test)


In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((70000,), (30000,), (70000,), (30000,))

XGBOOST CLASSIFIER

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os

file = "obfuscation"

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(file), "..")))

from obfuscation.obfuscation import obfuscate, deobfuscate
from scipy.sparse import csr_matrix

# def convert_to_strings(feature_matrix):
#     """ Converts a sparse or dense matrix to a list of strings. """
#     if isinstance(feature_matrix, csr_matrix):
#         feature_matrix = feature_matrix.toarray()  # Convert sparse to dense

#     return [" ".join(map(str, row)) for row in feature_matrix]  # Convert row to string

# # Convert training and testing features to strings
# X_train_text = convert_to_strings(X_train_features)
# X_test_text = convert_to_strings(X_test_features)

# Now apply obfuscation
# X_train_obfuscated = [obfuscate(text)[0] for text in X_train]
# X_test_obfuscated = [obfuscate(text)[0] for text in X_test]

vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1))
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

print(X_train_transformed.shape, X_test_transformed.shape)

# Train model
model2 = XGBClassifier(n_estimators=100, random_state=42)
model2.fit(X_train_transformed, y_train)

# Predict on obfuscated test data
y_pred_obfuscated = model2.predict(X_test_transformed)

# Deobfuscate predictions if necessary (not needed for numerical labels)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_obfuscated)
print(f"Model Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_obfuscated))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_obfuscated))


(70000, 39) (30000, 39)
Model Accuracy: 0.5841

Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.45      0.41      5982
           1       0.61      0.51      0.55      6060
           2       0.38      0.43      0.41      5978
           3       1.00      1.00      1.00      5994
           4       0.63      0.53      0.58      5986

    accuracy                           0.58     30000
   macro avg       0.60      0.58      0.59     30000
weighted avg       0.60      0.58      0.59     30000


Confusion Matrix:
[[2686  297 2682    1  316]
 [ 904 3077  830    1 1248]
 [2741  330 2590    1  316]
 [   0    0    0 5994    0]
 [ 759 1336  710    4 3177]]


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os

file = "obfuscation"

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(file), "..")))

from obfuscation.obfuscation import obfuscate, deobfuscate
from scipy.sparse import csr_matrix

# def convert_to_strings(feature_matrix):
#     """ Converts a sparse or dense matrix to a list of strings. """
#     if isinstance(feature_matrix, csr_matrix):
#         feature_matrix = feature_matrix.toarray()  # Convert sparse to dense

#     return [" ".join(map(str, row)) for row in feature_matrix]  # Convert row to string

# # Convert training and testing features to strings
# X_train_text = convert_to_strings(X_train_features)
# X_test_text = convert_to_strings(X_test_features)

# Now apply obfuscation
X_test_obfuscated = [obfuscate(text)[0] for text in X_test]

vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1))
X_all = np.concatenate((X_train, X_test_obfuscated), axis=0)  # Correct way

vectorizer.fit(X_all)

X_train_transformed = vectorizer.transform(X_train)
X_test_transformed = vectorizer.transform(X_test_obfuscated)

print(X_train_transformed.shape, X_test_transformed.shape)

# Train model
model2 = XGBClassifier(n_estimators=100, random_state=42)
model2.fit(X_train_transformed, y_train)

# Predict on obfuscated test data
y_pred_obfuscated = model2.predict(X_test_transformed)

# Deobfuscate predictions if necessary (not needed for numerical labels)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_obfuscated)
print(f"Model Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_obfuscated))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_obfuscated))


(70000, 129) (30000, 129)
Model Accuracy: 0.2022

Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.43      0.26      5982
           1       0.35      0.00      0.01      6060
           2       0.22      0.27      0.25      5978
           3       0.22      0.23      0.22      5994
           4       0.17      0.08      0.11      5986

    accuracy                           0.20     30000
   macro avg       0.23      0.20      0.17     30000
weighted avg       0.23      0.20      0.17     30000


Confusion Matrix:
[[2569   17 1696 1205  495]
 [2612   22 1809 1056  561]
 [2580   10 1643 1221  524]
 [3164    3  698 1355  774]
 [2617   10 1556 1327  476]]


WITH LSTM

In [55]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from collections import Counter

file = "obfuscation"

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(file), "..")))
from obfuscation.obfuscation import obfuscate, deobfuscate


# Step 1: Apply obfuscation to test data
X_test_obfuscated = [obfuscate(text)[0] for text in X_test] 
print(X_test_obfuscated[:10]) # Ensure obfuscation works on raw text
all_text = list(df['Ciphertext']) + X_test_obfuscated  # Combine both
all_chars = "".join(all_text) 

# print(all_chars[:100])  # Print first 100 characters for inspection
char_counts = Counter(all_chars)
char_vocab = sorted(char_counts.keys())  # Unique characters
char_to_idx = {char: idx for idx, char in enumerate(char_vocab, start=1)}
# print(f"Vocabulary Size: {char_to_idx.keys()}")

# Function to convert ciphertext to sequence of numbers
def text_to_sequence(text, max_len=100):
    seq = [char_to_idx.get(char, 0) for char in text]
    seq = seq[:max_len] + [0] * (max_len - len(seq))  # Pad/truncate to max_len
    return seq

# Step 2: Convert obfuscated text into numerical sequences
X_train_seq = torch.tensor([text_to_sequence(text) for text in X_train], dtype=torch.long)
X_test_seq_obfuscated = torch.tensor([text_to_sequence(text) for text in X_test_obfuscated], dtype=torch.long)

# Step 3: Convert labels to tensors
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# # Convert all ciphertexts
# X_train_seq = torch.tensor([text_to_sequence(text) for text in X_train], dtype=torch.long)
# X_test_seq = torch.tensor([text_to_sequence(text) for text in X_test], dtype=torch.long)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)


['c\x81\x8d·\x8dµÛ\x9fu\x93u\x8bë¡×íÛÏ\xadÑug\x93oÝÛÃm\x9f\x91\x99gé\x93ica×\x89\x93§áÑggéáëÙcÉ£\x91aõ¥«\x9bÑ\x9f\x93£ÉË382404', 'Û\x7fÉÇßéÑÏßw\x93÷ï¥åçemScg\x9bÏ¯Ã÷\x9f¥\x8dç\xad¥©\x89[¡µiå\x8f\x97ÏÇa\x95·ñµ\x99\x81Ção±§oß£¯çmÑ\x8f\x9b¡¥ÑéÑßÍg\x7fñ·\x87030610', 'êpº\x84ì¸â\x9cºÆ\x82b®Òf\x8c¤þ\x82\x96Ô\x88Ì\x98æfØdjúîf¤xÎdºÈz¬\x88Ô\x82\x86\x8e®ÒÐ\x94ÌÆ\x8cÞ\x94ª\x9a¦ÜfÂjzäþª\x9c\x92¾ê\x96\x92¬Æh\x80þh\x9a¤¢h\x86\x8c\xa0pfÐ\x88677603', 'nâ¨nÜ\x9e\x80\x92º¨\xa0TÌÎÞxÀÚ\x98èæ¦\x90\xa0ÚæÈf¢\x82È\x96¬\x94\x80¨hØìÎ\\¦ª®\x9a\x9cÞÎÖ®¸f®jÈ\x98\x88âÎæøèì\xa0022709', 'úpd\x8c\x82\x8câàÈªjàlÚú\x84îèÌä\x9aâfÞ\xa0\x96z\x96xþ\x84à\x96Ò\x8c\x82¾øÈæÎ\x80î¤\x88ÀÔTÎªhîxÆ¢\x90xþxÎþ\x98¦ºTî\\ºpøÚä672917', '\x9fÝ\xadÙ\x99\x9b\x9b\x9b×ÝÃµÝÉ\xadßËi\x89\x81ëõu©ÍçuÓÍáÛßõ\x97å×\x89©meÍ\x87µ÷\x9dÍ\x93\x9b©«ëµ\x99\x95é¯Áu\x8fã\x9fÝ\x9dÇ172520', '\x9cºb\x98jTÀì¬¢¾\x86¾\x9elâøl`dx\x8cn¢¤TÚ\x82\x86êhî¬\x92¬\x86îªºøÂÚÂ¬\\húî\x80d\x9c\x92\x9c¾þÐÌþnÆ\x94z\x80Ìââ\x9e\x94è\x8cÚÈbèbþ325014', '·\x7f§\x97aÑ\x83\x9f\x8fÉ÷w\x95

In [62]:
import torch
from torch.utils.data import DataLoader, Dataset
from obfuscation.obfuscation import obfuscate  # Import the obfuscation function

class EncryptionDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.X[idx] is None or self.y[idx] is None:  # Debugging step
            print(f"Found None at index {idx}")
        return self.X[idx], self.y[idx]


batch_size = 32

train_dataset = EncryptionDataset(X_train_seq, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = EncryptionDataset(X_test_seq_obfuscated, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [63]:
import torch.nn as nn
import pickle

class CipherLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, dropout_prob=0.5):
        super(CipherLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        hidden = self.dropout(hidden[-1])  # Apply dropout before the fully connected layer
        out = self.fc(hidden)
        return out

# Model parameters
vocab_size = len(char_vocab) + 1  # Extra 1 for padding index
embed_dim = 128
hidden_dim = 256
num_classes = len(label_encoder.classes_)

# Initialize model
model = CipherLSTM(vocab_size, embed_dim, hidden_dim, num_classes, dropout_prob=0.5)

In [64]:
# model_path = r"C:\Users\Apurav\Downloads\hackTheFuture\Hack-the-Future\models\cipher_lstm_model.pth"
# model.load_state_dict(torch.load(model_path))

# # Set to evaluation mode
# model.eval()

# print("Model loaded successfully!")


In [65]:
print(f"X_train_seq contains None: {any(x is None for x in X_train_seq)}")
print(f"y_train_tensor contains None: {any(y is None for y in y_train_tensor)}")


X_train_seq contains None: False
y_train_tensor contains None: False


In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        # batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print loss every epoch
    # if (epoch + 1) % 1 == 0:
    #     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/3], Loss: 0.6378
Epoch [2/3], Loss: 0.6320
Epoch [3/3], Loss: 0.6309


In [68]:
import pickle

# Save the model
model_path = r"C:\Users\Apurav\Downloads\hackTheFuture\Hack-the-Future\models\cipher_lstm_model_after_obfuscation.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to C:\Users\Apurav\Downloads\hackTheFuture\Hack-the-Future\models\cipher_lstm_model_after_obfuscation.pth


In [69]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:

        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)

        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(batch_y.cpu().numpy())

# Compute Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

# Print Classification Report (Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_labels, predictions))

# Print Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predictions))


Test Accuracy: 0.1962

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5982
           1       0.21      0.90      0.34      6060
           2       0.06      0.00      0.00      5978
           3       0.00      0.00      0.00      5994
           4       0.43      0.07      0.12      5986

    accuracy                           0.20     30000
   macro avg       0.14      0.19      0.09     30000
weighted avg       0.14      0.20      0.09     30000


Confusion Matrix:
[[   0 5916    6    0   60]
 [ 594 5465    0    0    1]
 [   0 5907   14    0   57]
 [1011 4450  119    0  414]
 [ 987 4512   79    0  408]]


c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Apurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

With Sentence Transformer

In [35]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

model_path = r"C:\Users\Apurav\OneDrive\Desktop\Project_ML\Cold_email_generator\models\all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_path)


In [36]:
embeddings = embedding_model.encode(df_filtered['Ciphertext'].tolist(), batch_size=32, show_progress_bar=True)

X = np.array(embeddings)
y = df_filtered['algorithm_label']

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

In [37]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train model
model2 = XGBClassifier(n_estimators=100, random_state=42)
model2.fit(X_train_features, y_train)


# model = KNeighborsClassifier(n_neighbors=100)
# model.fit(X_train_features, y_train)

# Predict
y_pred = model2.predict(X_test_features)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))